In [ ]:
!pip install tensorflow-gpu

In [ ]:
#basic LIB
import numpy as np
import pandas as pd
import os
import warnings
import sys
from numpy import asarray
from numpy import zeros
#dataset
from keras.datasets import imdb
#Visulization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML,display
import random
#Preprosessing
import nltk
import gensim
from gensim.models import Word2Vec,Phrases
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import Counter
from wordcloud import WordCloud
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
from nltk import pos_tag
from nltk.corpus import wordnet

#Model Building
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Conv1D, MaxPool1D,GlobalMaxPooling1D, Dense, Dropout , LSTM,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import legacy

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Limit GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
print(tf.test.gpu_device_name())

In [ ]:
!pip install --upgrade nltk

In [ ]:
nltk.download('wordnet')  # download the corpus
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
from keras.datasets import imdb

In [ ]:
num_words = 10000 # Change this to adjust the vocabulary size
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=num_words)

# Decode the reviews back to English words
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


In [ ]:

# Decode all reviews in the training data
reviews = []
for review in train_data:
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in review]) # <pad> <Sos> <unk>
    reviews.append(decoded_review)

# Decode all reviews in the test data
for review in test_data:
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in review])
    reviews.append(decoded_review)

In [ ]:
# combine all labels
Labels = []
for label in train_labels:
    Labels.append(label)
test_reviews = []
for label in test_labels:
    Labels.append(label)

In [ ]:
df = pd.DataFrame({'review': reviews, 'sentiment': Labels})

In [ ]:
df['sentiment']=df['sentiment'].map({ 1 : 'positive',  0 : 'negative'})

In [ ]:
df.info()
#our data consist od 50K records with 2 columns 1 feature and 1 Label

In [ ]:
df.groupby(['sentiment']).describe()
#25K Positive and 25 K negative  thus our data is balanced and with 24698 , 24884 unique words for each
#and  3 , 5  Frequency
#let's combine them and see what's total uniqe value

In [ ]:
df['review'].nunique()
# 49582 unique word

In [ ]:
#let's check if there is duplicate values
duplicates = df[df.duplicated()]
print(f'Number of duplicate rows: {len(duplicates)}')
#yes there is duplicated values

In [ ]:
#let's check for null
df.isnull().sum()
#there is no null values at any column

In [ ]:
sns.countplot(x=df['sentiment']) #ploting distribution for easier understanding

In [ ]:
#let's see our dataframe
df.head(3)

In [ ]:
#let's see how data is looklike
random_index=random.randint(0,df.shape[0]-3)
for row in df[['review','sentiment']][random_index:random_index+3].itertuples():
    _,text,label=row
    class_name="Positive"
    if label==0:
        class_name="Negative"
    display(HTML(f"<h5><b style='color:red'>Text: </b>{text}</h5>"))
    display(HTML(f"<h5><b style='color:red'>Target: </b>{class_name}<br><hr></h5>"))
#data contain so much garbage needs to be cleaned

In [ ]:
positivedata = df[df['sentiment']== 'positive']
positivedata =positivedata['review']
negdata = df[df['sentiment']== 'negative']
negdata= negdata['review']

def wordcloud_draw(data, color, s):
    words = ' '.join(data)
    cleaned_word = " ".join([word for word in words.split() if (word not in ['movie', 'film'])])
    if not cleaned_word:
        cleaned_word = 'no_data'  # Set a default value if there are no words
    word_counts = Counter(cleaned_word.split())
    if not word_counts:
        word_counts = {'no_data': 1}  # Set a default value if there are no words
    wordcloud = WordCloud(background_color=color, width=2500, height=2000).generate_from_frequencies(word_counts)
    plt.imshow(wordcloud)
    plt.title(s)
    plt.axis('off')

plt.figure(figsize=[20,10])
plt.subplot(1,2,1)
wordcloud_draw(positivedata,'white','Most-common Positive words')

plt.subplot(1,2,2)
wordcloud_draw(negdata, 'white','Most-common Negative words')
plt.show()

In [ ]:
df.drop_duplicates(inplace=True)
if df.duplicated().sum()==0:
    print('Duplicates Removed')
else:
    print('there is still duplicates')

In [ ]:
colors=['#AB47BC','#6495ED']
plt.pie(df['sentiment'].value_counts(),labels=['Positive','Negative'],autopct='%.2f%%',explode=[0.01,0.01],colors=colors);
plt.title('Distribution of target')
plt.ylabel('Sentiment');
#data still balanced

In [ ]:
df['text_word_count']=df['review'].apply(lambda x:len(x.split()))

numerical_feature_cols=['text_word_count']


In [ ]:

plt.figure(figsize=(20,3))
for i,col in enumerate(numerical_feature_cols):
    plt.subplot(1,3,i+1)
    sns.histplot(data=df,x=col,bins=50,color='#6495ED')
    plt.title(f"Distribution of Various word counts")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,3))
for i,col in enumerate(numerical_feature_cols):
    plt.subplot(1,3,i+1)
    sns.histplot(data=df,x=col,hue='sentiment',bins=50)
    plt.title(f"Distribution of Various word counts with respect to target")
plt.tight_layout()
plt.show()

In [ ]:
nltk.download('stopwords')

In [ ]:
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')
stop=set(stopwords.words('english'))
#Tokenization of text
tokenizer=ToktokTokenizer()

In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

#Removing Emails
def remove_Emails(text):
    pattern=r'\S+@\S+'
    text=re.sub(pattern,'',text)
    return text

#Removing URLS
def remove_URLS(text):
    pattern=r'http\S+'
    text=re.sub(pattern,'',text)
    return text

#Removing special characters and white characters and punctuations
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#Removing numbers
def remove_numbers(text):
    pattern = r'\d+'
    text = re.sub(pattern, '', text)
    return text

#LOWERCASE
def lowercase_text(text):
    return text.lower()

#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

#removing the stopwords and conver to lower case
def remove_stopwords(text, is_lower_case=True):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
#Removing br word
def br_html(text):
    pattern=r'br'
    text = re.sub(pattern, '', text)
    return text

In [ ]:
df['review']=df['review'].apply(denoise_text)
df['review']=df['review'].apply(remove_URLS)
df['review']=df['review'].apply(remove_Emails)
df['review']=df['review'].apply(remove_special_characters)
df['review']=df['review'].apply(remove_numbers)
df['review']=df['review'].apply(lowercase_text)

#let's see how data is looklike after these prosessing
random_index=random.randint(0,df.shape[0]-3)
for row in df[['review','sentiment']][random_index:random_index+3].itertuples():
    _,text,label=row
    class_name="Positive"
    if label==0:
        class_name="Negative"
    display(HTML(f"<h5><b style='color:red'>Text: </b>{text}</h5>"))
    display(HTML(f"<h5><b style='color:red'>Target: </b>{class_name}<br><hr></h5>"))

In [ ]:
df['review']=df['review'].apply(br_html)

In [ ]:
df['review'][0]

In [ ]:
def get_wordnet_pos(tag):
    """
    Map POS tag to first character used by WordNetLemmatizer
    """
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default to noun if POS tag is unknown

In [ ]:
def preprocess_review(review):
    """
    Preprocess a single movie review by tokenizing and lemmatizing
    """
    lemmatizer = WordNetLemmatizer()
    # Tokenize the review into words
    words = nltk.word_tokenize(review)

    # Tag the words with their part-of-speech (POS) tags
    tagged_words = nltk.pos_tag(words)

    # Lemmatize the words using their POS tags
    lemmatized_words = []
    for word, tag in tagged_words:
        pos = get_wordnet_pos(tag)
        lemmatized_word = lemmatizer.lemmatize(word, pos=pos)
        lemmatized_words.append(lemmatized_word)

    # Join the lemmatized words back into a single string
    preprocessed_review = ' '.join(lemmatized_words)

    return preprocessed_review

In [ ]:
print ('BEFORE (remove_stopwords).. \n',df['review'][2])
df['review_lem']=df['review'].apply(preprocess_review)
print ('AFTER (remove_stopwords) .. \n',df['review_lem'][2])

In [ ]:
print ('BEFORE (remove_stopwords).. \n',df['review_lem'][2])
df['review_lem']=df['review_lem'].apply(remove_stopwords)
print ('AFTER (remove_stopwords) .. \n',df['review_lem'][2])

In [ ]:
df.to_csv('IMDB_Prosessed_V1.csv')

In [ ]:
df=pd.read_csv('IMDB_Prosessed_V1.csv')

In [ ]:
df.head()

In [ ]:
#parameter for OOV tokens  is set to out of vocab and padding
oov_tok = "<OOV>"
trunc_type='post'
padding_type='post'
max_seq_length = 10000

In [ ]:
# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words=10000,oov_token=oov_tok)

# Generate the word index dictionary
tokenizer.fit_on_texts(df['review_lem'])


In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(df['review_lem'], df['sentiment'].map({'positive': 1, 'negative': 0}), test_size=0.2, random_state=42)

In [ ]:
# Generate and pad the training sequences
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
x_train = pad_sequences(x_train, maxlen=max_seq_length, padding=padding_type, truncating=trunc_type)


In [ ]:
sentences = [review.split() for review in df['review_lem']]

In [ ]:
model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=8,sg=0)

In [ ]:
model.save('imdb_word2vec.model')

In [ ]:
# Load the trained Word2Vec model
model = gensim.models.Word2Vec.load('imdb_word2vec.model')

# Test the model on some sample words
word1 = 'good'
word2 = 'bad'
word3 = 'movie'
word4 = 'action'


In [ ]:
# Get the most similar words to word1 and word2
print("Most similar words to '{}' and '{}' are:".format(word1, word2))
print(model.wv.most_similar(positive=[word1], negative=[word2]))

# Check if word3 is present in the vocabulary
if word3 in model.wv.key_to_index:
    print("Word '{}' is present in the vocabulary".format(word3))
else:
    print("Word '{}' is not present in the vocabulary".format(word3))

# Check if word4 is present in the vocabulary
if word4 in model.wv.key_to_index:
    print("Word '{}' is present in the vocabulary".format(word4))
else:
    print("Word '{}' is not present in thevocabulary".format(word4))

In [ ]:
vocab_size = len(model.wv.key_to_index)
embedding_dim = 100

In [ ]:
# Generate word embeddings from the model
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for i, word in enumerate(model.wv.index_to_key):
    embedding_matrix[i] = model.wv[word]

In [ ]:
# Build the model
with tf.device('/GPU:0'):
    model_lstm = Sequential ([
            Embedding(input_dim=vocab_size,output_dim=embedding_dim,weights=[embedding_matrix],trainable=False),
            Bidirectional(LSTM(100, return_sequences=True, dropout=0.35, recurrent_dropout=0.35)),
            Conv1D(100, 5, activation='relu'),
            GlobalMaxPooling1D(),
            Dense(100, activation='relu'),
            Dropout(0.5),
            Dense(1, activation='sigmoid')
        ])

# Set the training parameters
    optimizer = Adam(learning_rate=0.001)
    model_lstm.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    # Print the model summary
# model_lstm.summary()


In [ ]:
model_lstm.summary()

In [ ]:
tf.keras.utils.plot_model(model_lstm,show_shapes=True)

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

# create a TensorBoard callback
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

In [ ]:
# Monitor memory usage during training
import psutil
import os

def print_memory_usage():
  process = psutil.Process(os.getpid())
  print(f"GPU memory usage: {process.memory_info().vms / 1e9:.2f} GB")


In [ ]:
# Train the model
from sklearn.metrics import f1_score
with tf.device('/GPU:0'):
    early_stopping = EarlyStopping(patience=3)
    print_memory_usage()
    # Reduce the batch size or the number of epochs to reduce memory usage.
    # Reduce the batch size or the number of epochs to reduce memory usage.
    history_lstm = model_lstm.fit(x_train, y_train, epochs=1, batch_size=32, verbose=1, validation_split=0.25, callbacks=[early_stopping, tensorboard_callback])
    print_memory_usage()

GPU memory usage: 15.89 GB
363/930 [==========>...................] - ETA: 7:00:49 - loss: 0.6097 - accuracy: 0.6596

In [ ]:
_, accuracy = model_lstm.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy}")

In [ ]:
y_pred = model_lstm.predict(x_test)
# Convert the predicted probabilities to binary labels
y_pred = (y_pred > 0.5).astype(int)
# Compute the F1 score
f1 = f1_score(validation_labels, y_pred)
# Print the F1 score
print("F1 score:", f1)

In [ ]:
def plot_loss_curves(history):

    '''
      returns seperate loss curves for training and validation metrics
    '''

    train_loss=history.history['loss']
    val_loss=history.history['val_loss']

    train_accuracy=history.history['accuracy']
    val_accuracy=history.history['val_accuracy']

    epochs=range(1,len(history.history['loss'])+1)
    plt.figure(figsize=(20,5))

    # plot loss data
    plt.subplot(1,2,2)
    plt.plot(epochs,train_loss,label="training_loss")
    plt.plot(epochs,val_loss,label="validation_loss")
    plt.title("Loss curves",size=20)
    plt.xlabel('epochs',size=20)
    plt.ylabel('loss',size=20)
    plt.legend(fontsize=15);
    # plt.show()


    # plot accuracy data
    plt.subplot(1,2,1)
    plt.plot(epochs,train_accuracy,label="training_acc")
    plt.plot(epochs,val_accuracy,label="validation_acc")
    plt.title("Accuracy curves",size=20)
    plt.xlabel('epochs',size=20)
    plt.ylabel('Accuracy',size=20)
    plt.tight_layout()
    plt.legend(fontsize=15);



    plt.title('Model Performance Curves')


plot_loss_curves(history_lstm)